<a href="https://colab.research.google.com/github/miiimee/Eye_disease_detection/blob/main/eye_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
# df = pd.read_csv('/content/dataset.zip')
# Upload the dataset folder as a zip file
uploaded = files.upload()

# Create a directory to store the uploaded dataset
os.makedirs('dataset', exist_ok=True)

# Unzip the uploaded file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('dataset')  # Extract to a folder named 'dataset'

# Check the extracted folder structure
os.listdir('dataset')

KeyboardInterrupt: 

In [ ]:
# Create directories for each class
os.makedirs('dataset/cataract', exist_ok=True)
os.makedirs('dataset/diabetic_retinopathy', exist_ok=True)
os.makedirs('dataset/glaucoma', exist_ok=True)
os.makedirs('dataset/normal', exist_ok=True)

# Move uploaded files to the appropriate directories
# You can manually specify which files go into which directory
# For example:
# os.rename('uploaded_file_name.jpg', 'dataset/cataract/uploaded_file_name.jpg')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
dataset_path = 'dataset'  # Path to the dataset folder

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 20% of data for validation
)

# Data generator for validation (no augmentation, only rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training',  # Use the training subset
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='validation',  # Use the validation subset
    shuffle=False
)

In [ ]:
# Build the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(5, activation='softmax'))  # 4 classes: cataract, diabetic_retinopathy, glaucoma, normal

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_generator.samples

In [ ]:
validation_generator.samples

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50  # You can increase the number of epochs for better training
)

In [ ]:
# Step 1: Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Step 2: Plot training and validation accuracy and loss
history_df = pd.DataFrame(history.history)

# Set the style for seaborn
import seaborn as sns
sns.set(style="whitegrid")

# Plotting Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.lineplot(data=history_df[['accuracy', 'val_accuracy']], palette='tab10')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])

# Plotting Loss
plt.subplot(1, 2, 2)
sns.lineplot(data=history_df[['loss', 'val_loss']], palette='tab10')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])

plt.tight_layout()
plt.show()

In [ ]:
model.save('eye_disease_cnn_model.h5')

In [ ]:
# Step 1: Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Step 2: Plot training and validation accuracy and loss
history_df = pd.DataFrame(history.history)

# Set the style for seaborn
import seaborn as sns
sns.set(style="whitegrid")

# Plotting Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.lineplot(data=history_df[['accuracy', 'val_accuracy']], palette='tab10')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])

# Plotting Loss
plt.subplot(1, 2, 2)
sns.lineplot(data=history_df[['loss', 'val_loss']], palette='tab10')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

# Step 1: Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Step 2: Plot training and validation accuracy and loss
# Extracting the history of training
history_dict = history.history

# Create a DataFrame for easier plotting
import pandas as pd

history_df = pd.DataFrame(history_dict)

# Set the style for seaborn
sns.set(style="whitegrid")

# Plotting Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.lineplot(data=history_df[['accuracy', 'val_accuracy']], palette='tab10')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])

# Plotting Loss
plt.subplot(1, 2, 2)
sns.lineplot(data=history_df[['loss', 'val_loss']], palette='tab10')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])

plt.tight_layout()
plt.show()

In [ ]:
# from PIL import Image
# # Preprocess the test image
# def preprocess_image(image_path):
#     img = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format
#     img = img.resize((150, 150))  # Resize to match the model's input size
#     img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
#     img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
#     return img_array

# # Preprocess the uploaded image
# test_image = preprocess_image('/content/1084_right.jpg')

In [ ]:
# # Make predictions
# predictions = model.predict(test_image)

# # Get the class with the highest probability
# predicted_class_index = np.argmax(predictions)
# class_labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']  # Adjust based on your dataset
# predicted_class = class_labels[predicted_class_index]

# print(f"Predicted Class: {predicted_class}")
# print(f"Class Probabilities: {predictions}")

In [ ]:
# import matplotlib.pyplot as plt

# # Display the test image
# plt.imshow(Image.open(test_image_path))
# plt.title(f"Predicted Class: {predicted_class}")
# plt.axis('off')
# plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Path to the image you want to test
image_path = '/content/1084_right.jpg'  # Replace with your image path

# Load and preprocess the image
test_image = load_and_preprocess_image(image_path)

# Make predictions
predictions = model.predict(test_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions, axis=1)[0]

# Map the predicted class index to the class label
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

# Display the result
print(f"Predicted class: {predicted_class_label}")

# Optional: Display the image being tested
plt.imshow(load_img(image_path, target_size=(150, 150)))
plt.axis('off')  # Hide axes
plt.title(f"Predicted: {predicted_class_label}")
plt.show()